![Variedades americanas](paper.jpg)

# Genómica poblacional del Mango en América

### Prácticas de Bioinformática para el curso de Genómica de la Conservación 2022 de Ciencias Agrogenómicas
### Asistente: Eddy Mendoza (eddyjmeg@gmail.com)

### Tu nombre:

En esta segunda parte, exploraremos la variación genética y la estructura de las poblaciones usando los SNPs que llamamos anteriormente. 

Como habrás visto, la carpeta de salida tiene varios archivos. Nosotros usaremos <snps.vcf> y <.plink>

El software que usaremos en esta ocasión: 

    conda install -c bioconda plink
    conda install -c bioconda admixture

### Análisis de componentes principales

In [1]:
# Primero necesitamos eliminar los SNPs que se encuentran ligados, estos nos pueden meter ruido.
! sed 's/Vanilla_planifolia_SAGARPA_Sc//g' populations.snps.vcf > snps.vcf
#Este comando los busca y los manda a otro archivo
! plink --vcf snps.vcf --allow-extra-chr --indep-pairwise 50 10 0.1 --out linked

# Ahora tomaremos el set original y le restaremos los SNPs ligados
! echo "___________________________________________"
! plink --vcf snps.vcf --allow-extra-chr --extract linked.prune.in --make-bed --mind --out pruned

PLINK v1.90b6.21 64-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to linked.log.
Options in effect:
  --allow-extra-chr
  --indep-pairwise 50 10 0.1
  --out linked
  --vcf snps.vcf

772642 MB RAM detected; reserving 386321 MB for main workspace.
--vcf: linked-temporary.bed + linked-temporary.bim + linked-temporary.fam
written.
59 variants loaded from .bim file.
37 people (0 males, 0 females, 37 ambiguous) loaded from .fam.
Ambiguous sex IDs written to linked.nosex .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 37 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.893724.
59 variants and 37 people pass filters and QC.
Not

In [2]:
# Ahora le pediremos a PLINK que haga por nosotros un PCA
! plink --bfile pruned --allow-extra-chr --pca --out pca

PLINK v1.90b6.21 64-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to pca.log.
Options in effect:
  --allow-extra-chr
  --bfile pruned
  --out pca
  --pca

772642 MB RAM detected; reserving 386321 MB for main workspace.
42 variants loaded from .bim file.
31 people (0 males, 0 females, 31 ambiguous) loaded from .fam.
Ambiguous sex IDs written to pca.nosex .
Using up to 71 threads (change this with --threads).
Before main variant filters, 31 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.963134.
42 variants and 31 people pass filters and QC.
Note: No phenotypes present.
Relationship matrix calculation complete.
--pca: Results saved to pca.eigenval and pca.eig

### ADMIXTURE

In [19]:
! bash admixture.sh

### Ahora crea una sesión de R para graficar los resultados. Vamos a aprovechar a ponerle el nombre de las poblacioneas a cada individuo.  

In [21]:
pip install pandas

     |████████████████████████████████| 11.3 MB 8.0 MB/s            
     |████████████████████████████████| 15.7 MB 46.5 MB/s            
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import os
metadata = pd.read_csv("metadata_sra.tsv", sep='\t', header=None)
pca = pd.read_csv("pca.eigenvec", sep=' ', header=None)
clusters = pd.merge(metadata, pca, left_on=3, right_on=0)